In [1]:
# !pip install -q openai
# !pip install -q dotenv

In [2]:
import os
import openai
from dotenv import load_dotenv, find_dotenv

# 读取本地/项目的环境变量。

# find_dotenv()寻找并定位.env文件的路径
# load_dotenv()读取该.env文件，并将其中的环境变量加载到当前的运行环境中  
# 如果你设置的是全局的环境变量，这行代码则没有任何作用。
_ = load_dotenv(find_dotenv())

# 获取环境变量 OPENAI_API_KEY
openai.api_key = os.environ['OPENAI_API_KEY']

In [3]:
# 一个封装 OpenAI 接口的函数，参数为 Prompt，返回对应结果
def get_completion(prompt, model="gpt-3.5-turbo"):
    '''
    prompt: 对应的提示词
    model: 调用的模型，默认为 gpt-3.5-turbo(ChatGPT)，有内测资格的用户可以选择 gpt-4
    '''
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # 模型输出的温度系数，控制输出的随机程度
    )
    # 调用 OpenAI 的 ChatCompletion 接口
    return response.choices[0].message["content"]

In [4]:
essay_text = f"""
灌溉农田每年需要大量的水投入，对全球粮食生产至关重要。实际蒸散发(AET)是农田用水的一个主要指标，
已经开发了几种遥感产品来量化全球尺度的AET。在本研究中，我们在谷歌Earth Engine云计算环境下，
利用MODIS蒸散发产品(2001-2018年)估算实际AET、潜在ET (PET)和降水率(PP)的全球趋势。然后，
我们引入了一个基于AET、PET和PP估计的组合的新指数——蒸散发预警指数(ETWI)，我们用它来评估观测到的AET趋势的可持续性。
研究表明，虽然AET在全球自然土地上没有显著变化，但在全球农田上显著增加(+14%±5%)。全球农田的平均ETWI为−0.40±0.25，
这主要是由AET的极端趋势驱动的，超过了PET和PP的趋势。此外，在全球范围内，水和能源受限地区的趋势表现为，
虽然AET和PET在水和能源受限地区都没有显著的趋势，但PP在能源受限地区的增长趋势大于水受限地区。
在国家层面平均农田ETWI趋势进一步揭示了泰国、巴西和中国农田用水的不可持续趋势。研究还发现，
这些地区在净初级产量(NPP)和太阳诱导荧光(SIF)方面的增长幅度最大，表明最近粮食产量的增长可能依赖于不可持续的水投入。
在全球范围内，与其他作物类型相比，灌溉玉米与不可持续的AET趋势有关。本文提出一个在线开放获取应用程序，
旨在实现近实时监测并提高对全球水消耗和可用性的了解。
"""

In [5]:
# 提示指令
prompt = f"""
你的任务是从论文摘要中生成一个更短的总结。

请对三个反引号之间的摘要文本进行概括，最多30个词汇。

总结: ```{essay_text}```
"""

# 调用 GPT3.5 回复指令
response = get_completion(prompt)
print(response)

全球农田的实际蒸散发(AET)显著增加，导致农田用水不可持续。水和能源受限地区的趋势表明，能源受限地区的降水率增长更快。泰国、巴西和中国的农田用水也呈不可持续趋势。灌溉玉米与不可持续的AET趋势有关。


In [7]:
def get_completion_from_messages(messages, 
                                 model="gpt-3.5-turbo", 
                                 temperature=0, 
                                 max_tokens=500):
    '''
    封装一个支持更多参数的自定义访问 OpenAI GPT3.5 的函数

    参数: 
    messages: 这是一个消息列表，每个消息都是一个字典，包含 role(角色）和 content(内容)。角色可以是'system'、'user' 或 'assistant’，内容是角色的消息。
    model: 调用的模型，默认为 gpt-3.5-turbo(ChatGPT)，有内测资格的用户可以选择 gpt-4
    temperature: 这决定模型输出的随机程度，默认为0，表示输出将非常确定。增加温度会使输出更随机。
    max_tokens: 这决定模型输出的最大的 token 数。
    '''
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, # 这决定模型输出的随机程度
        max_tokens=max_tokens, # 这决定模型输出的最大的 token 数
    )
    return response.choices[0].message["content"]

In [9]:
messages =  [  
{'role':'system', 
 'content':'你是一个助理， 并以三国演义的风格作出回答。'},    
{'role':'user', 
 'content':'就大学生活为主题给我写一些标题'},  
] 
response = get_completion_from_messages(messages, temperature=1)
print(response)

1. 江湖初闻：大学生活的开端
2. 刀光剑影下的大学江湖
3. 枪林弹雨中的求学之路
4. 红颜薄命：大学生活的甜酸苦辣
5. 神机妙算：如何在大学江湖中立于不败之地
6. 谋略策划：大学生活中的智慧与决断
7. 机智过人：百战百胜的大学生活秘籍
8. 衣食住行：在大学江湖中学会生存之道
9. 心怀天下：大学生活中的社会责任与担当
10. 忠义仁勇：大学生活中的友善与勇气


In [10]:
def get_completion_and_token_count(messages, model="gpt-3.5-turbo", temperature=0, max_tokens=500):
    """
    使用 OpenAI 的 GPT-3 模型生成聊天回复，并返回生成的回复内容以及使用的 token 数量。

    参数:
    messages: 聊天消息列表。
    model: 使用的模型名称。默认为"gpt-3.5-turbo"。
    temperature: 控制生成回复的随机性。值越大，生成的回复越随机。默认为 0。
    max_tokens: 生成回复的最大 token 数量。默认为 500。

    返回:
    content: 生成的回复内容。
    token_dict: 包含'prompt_tokens'、'completion_tokens'和'total_tokens'的字典，分别表示提示的 token 数量、生成的回复的 token 数量和总的 token 数量。
    """
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, 
        max_tokens=max_tokens,
    )

    content = response.choices[0].message["content"]
    
    token_dict = {
        'prompt_tokens':response['usage']['prompt_tokens'],
        'completion_tokens':response['usage']['completion_tokens'],
        'total_tokens':response['usage']['total_tokens'],
    }

    return content, token_dict

In [11]:
messages =  [  
{'role':'system', 
 'content':'你是一个助理， 并以三国演义的风格作出回答。'},    
{'role':'user', 
 'content':'就大学生活为主题给我写一些标题'},  
] 
response, token_dict = get_completion_and_token_count(messages)
print(response)

1. 学海无涯，大学生活启航
2. 书香门第，大学生活如诗如画
3. 知行合一，大学生活修身养性
4. 知识的战场，大学生活奋发向前
5. 知己知彼，大学生活交友无限
6. 挥洒青春，大学生活激情四溢
7. 挑战自我，大学生活勇往直前
8. 智慧的殿堂，大学生活智慧闪耀
9. 拼搏奋斗，大学生活成就辉煌
10. 知识的瑰宝，大学生活收获满满


In [12]:
print(token_dict)

{'prompt_tokens': 50, 'completion_tokens': 213, 'total_tokens': 263}


In [13]:
# 列出当前可用模型
openai.Model.list()

<OpenAIObject list at 0x1829211b310> JSON: {
  "object": "list",
  "data": [
    {
      "id": "whisper-1",
      "object": "model",
      "created": 1677532384,
      "owned_by": "openai-internal",
      "permission": [
        {
          "id": "modelperm-KlsZlfft3Gma8pI6A8rTnyjs",
          "object": "model_permission",
          "created": 1683912666,
          "allow_create_engine": false,
          "allow_sampling": true,
          "allow_logprobs": true,
          "allow_search_indices": false,
          "allow_view": true,
          "allow_fine_tuning": false,
          "organization": "*",
          "group": null,
          "is_blocking": false
        }
      ],
      "root": "whisper-1",
      "parent": null
    },
    {
      "id": "babbage",
      "object": "model",
      "created": 1649358449,
      "owned_by": "openai",
      "permission": [
        {
          "id": "modelperm-49FUp5v084tBB49tC4z8LPH5",
          "object": "model_permission",
          "created": 166908

: 